In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from scipy import spatial
import scipy.constants
from scipy.constants import pi
import logging
import traceback
import time
import glob
logging.basicConfig(level=logging.INFO)

In [ ]:
def within(circle,point):
    if np.sqrt(np.dot(circle[0:2]-point,circle[0:2]-point))<circle[2]:
        return True
    else:
        return False

In [ ]:
def nearest_cell(circles,centroids):
    cells_xy = circles[0][:,0:2]
    aggrs_xy = np.array(centroids)
    d = spatial.distance_matrix(cells_xy,aggrs_xy)
    cells_idx = []
    for id in range(d.shape[1]):
        i = np.argmin(d[:,id])
        if within(circles[0][i,:],aggrs_xy[id,:]):
            cells_idx.append(np.array([i,id]))#cell id, aggregate id
    cells_idx = pd.DataFrame(np.array(cells_idx),columns=["circle_index","agg_index"])
    return cells_idx

In [ ]:
def cell_ids(circles,cell_labels):
    circles[0][:,0] = np.round(circles[0][:,0],decimals=-2)
    cdf = pd.DataFrame(circles[0][:,0:3], columns=['x', 'y','D_pix'])
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    cdf['x'][0:4]=1200
    cdf['x'][4:8]=1500
    cdf['x'][8:12]=1800
    cdf['x'][12:16]=2100
    cdf['x'][16:20]=2400
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    cdf['y'][0:4]=np.array([700,1000,1300,1600])
    cdf['y'][4:8]=np.array([700,1000,1300,1600])
    cdf['y'][8:12]=np.array([700,1000,1300,1600])
    cdf['y'][12:16]=np.array([700,1000,1300,1600])
    cdf['y'][16:20]=np.array([700,1000,1300,1600])
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
        
    idx = np.array(cdf.index)
    labels = cell_labels
    labels['circle_index']=idx
    return idx, cdf,labels
    

In [ ]:
p_raw = '../data/input'
p_work = '../data/output'
ncells=20
D=45
A_mm2=D**2*pi/4

t=[]
labels_t = []
matches_t = []
areas_t = []
circles_t = []
D_t = []
for f in os.listdir(p_raw):
    if '.csv' in f:
        cell_labels = pd.read_csv(os.path.join(p_raw,f))
j = 0
images = glob.glob(os.path.join(p_raw,'*.jpg'))

In [ ]:
for f in np.sort(images):     
    param2 = 20
    param1 = 400
    minspace=270
    ksize=15

    method = cv2.HOUGH_GRADIENT
    logging.info(f)
    img = cv2.imread(f)
    b,g,r = cv2.split(img)       
    b = cv2.GaussianBlur(b,(ksize,ksize),1)   
    retb,bt = cv2.threshold(b,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    h,s,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
    thresh = cv2.adaptiveThreshold(v,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,int(int(minspace)*2+1),35)     
    plt.imshow(thresh)
    plt.show()
    bt = bt-thresh
    plt.imshow(bt)
    plt.show()
    bt_cnt, hierarchy = cv2.findContours(image=bt, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    cnt = max(bt_cnt, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(cnt)
    x = x-20
    y = y-20
    w = w+40
    h = h+40
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 15)
    plt.imshow(img)
    plt.show()
    bt = bt[y:y+h, x:x+w]
    img = img[y:y+h, x:x+w,:]
    try:
        _,_,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
        retv,thresh = cv2.threshold(v,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))
        try:
            #adaptive parameter selection
            outer_max  = 5
            ot = 0
            while circles.shape[1]!=ncells and ot<outer_max:
                itermax = 40        
                it = 0
                while circles.shape[1]!=ncells and it<itermax:
                    if circles.shape[1]<ncells:
                        param1=param1-1
                        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                    if circles.shape[1]>ncells:
                        param1=param1+1
                        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                    it=it+1
                itermax = param2-2        
                it = 0
                while circles.shape[1]!=ncells and it<itermax:
                    if circles.shape[1]<ncells:
                        param2=param2-1
                        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                    if circles.shape[1]>ncells:
                        param2=param2+1
                        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                    it=it+1                
                itermax = 10        
                it = 0
                while circles.shape[1]!=ncells and it<itermax:
                    if circles.shape[1]<ncells:
                        minspace=minspace-2
                        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                    if circles.shape[1]>ncells:
                        minspace=minspace+2
                        circles = cv2.HoughCircles(bt,method,2,minspace,param1=param1,param2=param2,minRadius=int(minspace/2),maxRadius=int(minspace/2+20))

                    it=it+1
                ot=ot+1

            circles = np.uint16(np.around(circles))
            for i in circles[0,:]:
                # draw the outer circle
                cv2.circle(img,(i[0],i[1]),i[2],(0,0,255),5)  

            D_pix = 2*np.median(circles[0,:,2])
            A_pix = D_pix**2*pi/4
            A_convert = A_mm2/A_pix
            idx,cdf,labels = cell_ids(circles,cell_labels)

            contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
            # draw contours on the original image
            cv2.drawContours(image=img, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=5, lineType=cv2.LINE_AA)

            centers = []
            A = []
            for c in contours:
                # compute the center of the contour
                M = cv2.moments(c)
                if M["m00"]>0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    centers.append(np.array([cX,cY]))
                    A.append(M["m00"]*A_convert)

            cells_idx = nearest_cell(circles,centers)
            labels = labels.merge(cells_idx, how="outer", on="circle_index")

            #maps areas to cells
            A_agg = np.zeros(len(labels["agg_index"].values))*np.nan
            for idx, agid in enumerate(labels["agg_index"].values):
                if not np.isnan(agid): A_agg[idx] = A[int(agid)]

            labels["area"] = A_agg

            if not os.path.isdir(p_work): os.mkdir(os.path.join(p_work))
            fout = os.path.join(p_work,f.split('.')[2].split('/')[3]+'_labeled.jpg')
            cv2.imwrite(fout,img)

            plt.imshow(img)
            plt.show()

            labels=labels.groupby(['Column','Row','Well','Sample','Rep']).aggregate(func=sum).drop(columns=['circle_index','agg_index'])
            matches_t.append(cells_idx)
            areas_t.append(A)
            circles_t.append(circles)
            labels_t.append(labels)            
            D_t.append(D_pix)
            t.append(pd.to_datetime("".join(f.split('.')[2].split('/')[3][4:21].split('_'))))
            j = j+1

        except Exception as e:
            print(e)
            plt.subplot(121)
            plt.imshow(bt)
            plt.subplot(122)
            plt.imshow(img)
            plt.show()
    except Exception as e:
        print(e)
        plt.imshow(bt)
        plt.show()

In [43]:
labels_t=[labels_t[i] for i in np.argsort(t)]
t = np.sort(t)
t = t-np.array(min(t))
df = labels_t[0]
df = df.rename(columns={"area": str(t[0])})
for i in range(1,len(labels_t)):
    df=df.merge(labels_t[i],how='outer',on=['Column','Row','Well','Sample','Rep'])
    df = df.rename(columns={"area": str(t[i])})
print(os.path.join(p_work,'processed_aggregates.csv'))
df.to_csv(os.path.join(p_work,'processed_aggregates.csv'))

../data/output/processed_aggregates.csv
